In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip setuptools wheel
!{sys.executable} -m pip install --upgrade cmake 
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install --ignore-installed PyYAML
!{sys.executable} -m pip install --ignore-installed --upgrade "dask[complete]"
!{sys.executable} -m pip install --ignore-installed dask-sql

In [4]:
import pyarrow as pa
import pyarrow.parquet as pq
import dask
from dask import dataframe as dd
from dask.distributed import Client
import os

In [5]:
client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 7.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34929,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 7.70 GiB
Comm: tcp://127.0.0.1:34077,Total threads: 1
Dashboard: http://127.0.0.1:46735/status,Memory: 1.93 GiB
Nanny: tcp://127.0.0.1:33851,


In [6]:
customer = dd.read_parquet(os.path.join("data", "customer"), engine='pyarrow')
region = dd.read_parquet(os.path.join("data", "region"), engine='pyarrow')
nation = dd.read_parquet(os.path.join("data", "nation"), engine='pyarrow')
lineitem = dd.read_parquet(os.path.join("data", "lineitem"), engine='pyarrow')
orders = dd.read_parquet(os.path.join("data", "orders"), engine='pyarrow')
part = dd.read_parquet(os.path.join("data", "part"), engine='pyarrow')
partsupp = dd.read_parquet(os.path.join("data", "partsupp"), engine='pyarrow')
supplier = dd.read_parquet(os.path.join("data", "supplier"), engine='pyarrow')

In [7]:
customer.compute()

,c_custkey,c_name,c_address,c_phone,c_acctbal,c_mktsegment,c_comment,c_nationkey
28,29,Customer#000000029,"sJ5adtfyAkCK63df2,vF25zyQMVYE34uh",10-773-203-7342,7618.27,FURNITURE,its after the carefully final platelets x-ray ...,0
47,48,Customer#000000048,0UU iPhBupFvemNB,10-508-348-5882,3792.50,BUILDING,"re fluffily pending foxes. pending, bold plate...",0
72,73,Customer#000000073,8IhIxreu4Ug6tt5mog4,10-473-439-3214,4288.50,BUILDING,"usual, unusual packages sleep busily along the...",0
75,76,Customer#000000076,"m3sbCvjMOHyaOofH,e UkGPtqc4",10-349-718-3044,5745.33,FURNITURE,pecial deposits. ironic ideas boost blithely a...,0
79,80,Customer#000000080,"K,vtXp8qYB",10-267-172-7101,7383.53,FURNITURE,tect among the dependencies. bold accounts eng...,0
...,...,...,...,...,...,...,...,...
149889,149890,Customer#000149890,I53MLT9cywhdNANSfUzwJ,34-256-916-5025,-426.31,AUTOMOBILE,"he special ideas. regular, special packages sl...",24
149922,149923,Customer#000149923,gKvVWoaM ZqupFEJcY7YQVkoPus,34-856-650-2786,9568.43,BUILDING,d courts print alongside of the closely pendin...,24
149964,149965,Customer#000149965,0sO4zG8Yv2K,34-986-629-4469,2558.78,FURNITURE,"riously. final, bold accounts use fluffily abo...",24
149967,149968,Customer#000149968,e8w5DuzdTXUktKd,34-979-886-1763,1564.09,AUTOMOBILE,along the pending deposits. carefully ironic ...,24


In [8]:
from dask_sql import Context

c = Context()
c.create_table("customer", customer, persist=True)
c.create_table("region", region, persist=True)
c.create_table("nation", nation, persist=True)
c.create_table("lineitem", lineitem, persist=True)
c.create_table("orders", orders, persist=True)
c.create_table("part", part, persist=True)
c.create_table("partsupp", partsupp, persist=True)
c.create_table("supplier", supplier, persist=True)

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
result = c.sql("""
select
    l_orderkey,
    sum(l_extendedprice * (1 - l_discount)) as revenue,
    o_orderdate,
    o_shippriority
from
    customer,
    orders,
    lineitem
where
    c_mktsegment = 'BUILDING'
    and c_custkey = o_custkey
    and l_orderkey = o_orderkey
    and o_orderdate < date '1995-03-15'
    and l_shipdate > date '1995-03-15'
group by
    l_orderkey,
    o_orderdate,
    o_shippriority
order by
    revenue desc,
    o_orderdate
limit 10;
""")
result.compute()